In [1]:
import spacy
import requests
!python -m spacy download en_core_web_md
 
nlp = spacy.load("en_core_web_md")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 12.6 MB/s eta 0:00:00m eta 0:00:010:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


## Función que realiza la llamada al api

Utilizamos el servicio http://api.openweathermap.org para consultar el tiempo a partir del nombre de una ciudad.

### Insertamos nuestra key del api

In [2]:
#api_key = "66f690904c3f75b3e102e75993bdc5a1"

In [3]:
def get_weather(city_name):
    api_url = "http://api.openweathermap.org/data/2.5/weather?q={}&appid={}".format(city_name, api_key)
 
    response = requests.get(api_url)
    response_dict = response.json()
 
    weather = response_dict["weather"][0]["description"]
 
    if response.status_code == 200:
        return weather
    else:
        print('[!] HTTP {0} calling [{1}]'.format(response.status_code, api_url))
        return None

## Definimos el patrón para solicitar el tiempo

Algunos ejemplos podrían ser:

- Weather conditions in Alicante
- Weather conditions in Madrid

In [16]:
weather = nlp("Weather Conditions in city")

print(weather.similarity(nlp("Weather Conditions in Alicante")))
print(weather.similarity(nlp("El tiempo en Alicante")))
print(weather.similarity(nlp("Weather in Alicante")))

0.9322947770940994
0.09518664217966936
0.8891195740185529


## Función que comprueba el tiempo

Utilizamos la función similarity para comprobar la similitud con la intención e identificamos localizaciones en el texto.

In [26]:
def chatbot(query):
    print("chatbot query:", query)
    statement = nlp(query)
    print("chatbot statement:", statement)
    min_similarity = 0.75
 
    if weather.similarity(statement) >= min_similarity:
        print("ents:", statement.ents)
        for ent in statement.ents:
            print("ent:", ent)
            if ent.label_ == "GPE": # GeoPolitical Entity
                city = ent.text
         
                city_weather = get_weather(city)
                if city_weather is not None:
                    return "In " + city +", the current weather is: " + city_weather
                else:
                    return "Something went wrong."
            else:
                return "You need to tell me a city to check."
       
    else:
        return "Sorry I don't understand that. Please rephrase your statement."

## Iniciamos el chatbot 

Ejecutamos tantas veces como queramos el chatbot.

In [32]:
print("Hi! I am Windy a weather bot.........")
statement = input("How can I help you ?\n")
 
response = chatbot(statement)
print(response)

Hi! I am Windy a weather bot.........
How can I help you ?
Weather conditions in Madrid
chatbot query: Weather conditions in Madrid
chatbot statement: Weather conditions in Madrid
ents: (Madrid,)
ent: Madrid
In Madrid, the current weather is: clear sky


## Referencias

https://www.askpython.com/python/examples/chatbot-in-python-using-spacy